In [3]:
import matplotlib
import serial
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import threading
import queue
import time
import matplotlib.animation as animation

In [4]:
matplotlib.use('TkAgg')

SERIAL_PORT = '/dev/cu.usbserial-0001'
# SERIAL_PORT = 'dev/tty.'
BAUD_RATE = 250000
ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=0.05)

# Heatmap color map (optional)
# cmap = mcolors.LinearSegmentedColormap.from_list("custom", ["red", "yellow", "green"])

# Create figure and axis for plots
fig, (ax_graph) = plt.subplots(1, 1, figsize=(10, 6))

# ---- HEATMAP SETUP (COMMENTED OUT) ----
# matrix = np.zeros((5, 3))
# heatmap = ax_heatmap.imshow(matrix, cmap=cmap, vmin=0, vmax=3.3)
# cbar = plt.colorbar(heatmap, ax=ax_heatmap, label="Voltage (V)")
# ax_heatmap.set_xticks(np.arange(3))
# ax_heatmap.set_yticks(np.arange(5))
# ax_heatmap.set_xticklabels(["CH1", "CH2", "CH3"])
# ax_heatmap.set_yticklabels(["CH1", "CH2", "CH3", "CH4", "CH5"])
# ax_heatmap.set_title("5x3 Matrix Voltage Visualization")
# cell_texts = [[ax_heatmap.text(j, i, "", ha="center", va="center", color="black", fontsize=12)
#                for j in range(3)] for i in range(5)]

# Setup for real-time line plot
max_history = 200
time_series_data = [[[] for _ in range(3)] for _ in range(5)]  # 5x3 grid
x_vals = list(range(-max_history + 1, 1))

# Create line objects for each cell
lines = [[ax_graph.plot(x_vals, [0]*max_history, label=f"({i},{j})")[0] for j in range(3)] for i in range(5)]

# Configure the graph
ax_graph.set_ylim(0, 3.5)
ax_graph.set_xlim(-max_history + 1, 0)
ax_graph.set_title("Real-time Voltage Trends")
ax_graph.set_xlabel("Time (frames)")
ax_graph.set_ylabel("Voltage (V)")
ax_graph.legend(loc="upper right", fontsize="x-small", ncol=3)

# Queue for incoming serial data
data_queue = queue.Queue()

def update_plot(frame):
    if not data_queue.empty():
        new_matrix = data_queue.get()

        for i in range(5):
            for j in range(3):
                val = new_matrix[i][j]

                # Update rolling data
                if len(time_series_data[i][j]) >= max_history:
                    time_series_data[i][j].pop(0)
                time_series_data[i][j].append(val)

                # Update line
                lines[i][j].set_ydata(time_series_data[i][j])
                lines[i][j].set_xdata(x_vals[-len(time_series_data[i][j]):])

                # Optional: update heatmap text (commented)
                # cell_texts[i][j].set_text(f"{val:.2f}")

        # heatmap.set_data(new_matrix)

    # Return list of updated artists
    all_lines = [line for row in lines for line in row]
    # return [heatmap] + [text for row in cell_texts for text in row] + all_lines
    return all_lines

def read_arduino_data():
    while True:
        try:
            line = ser.readline().decode('utf-8', errors='ignore').strip()
            if "Matrix updated:" in line:
                new_matrix = np.zeros((5, 3))
                for i in range(5):
                    row_data = ser.readline().decode('utf-8', errors='ignore').strip().split(",")
                    if len(row_data) == 3:
                        new_matrix[i] = [float(val) for val in row_data]
                data_queue.put(new_matrix)
        except Exception as e:
            print(f"Serial read error: {e}")

# Background serial reader
data_thread = threading.Thread(target=read_arduino_data, daemon=True)
data_thread.start()

# Animation loop
ani = animation.FuncAnimation(fig, update_plot, interval=50, blit=False)
plt.tight_layout()
plt.show()


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
matplotlib.use('TkAgg')

SERIAL_PORT = '/dev/cu.usbserial-0001'
# SERIAL_PORT = 'dev/tty.'
BAUD_RATE = 250000
ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=0.05)

# Heatmap color map (optional)
cmap = mcolors.LinearSegmentedColormap.from_list("custom", ["red", "yellow", "green"])

# Create figure and axis for plots
fig, (ax_heatmap) = plt.subplots(1, 1, figsize=(10, 6))

# ---- HEATMAP SETUP (COMMENTED OUT) ----
matrix = np.zeros((5, 3))
heatmap = ax_heatmap.imshow(matrix, cmap=cmap, vmin=0, vmax=3.3)
cbar = plt.colorbar(heatmap, ax=ax_heatmap, label="Voltage (V)")
ax_heatmap.set_xticks(np.arange(3))
ax_heatmap.set_yticks(np.arange(5))
ax_heatmap.set_xticklabels(["CH1", "CH2", "CH3"])
ax_heatmap.set_yticklabels(["CH1", "CH2", "CH3", "CH4", "CH5"])
ax_heatmap.set_title("5x3 Matrix Voltage Visualization")
cell_texts = [[ax_heatmap.text(j, i, "", ha="center", va="center", color="black", fontsize=12)
               for j in range(3)] for i in range(5)]

# Setup for real-time line plot
max_history = 200
time_series_data = [[[] for _ in range(3)] for _ in range(5)]  # 5x3 grid
x_vals = list(range(-max_history + 1, 1))

# Create line objects for each cell
lines = [[ax_graph.plot(x_vals, [0]*max_history, label=f"({i},{j})")[0] for j in range(3)] for i in range(5)]

# Configure the graph
ax_graph.set_ylim(0, 3.5)
ax_graph.set_xlim(-max_history + 1, 0)
ax_graph.set_title("Real-time Voltage Trends")
ax_graph.set_xlabel("Time (frames)")
ax_graph.set_ylabel("Voltage (V)")
ax_graph.legend(loc="upper right", fontsize="x-small", ncol=3)

# Queue for incoming serial data
data_queue = queue.Queue()

def update_plot(frame):
    if not data_queue.empty():
        new_matrix = data_queue.get()

        for i in range(5):
            for j in range(3):
                val = new_matrix[i][j]

                # Update rolling data
                if len(time_series_data[i][j]) >= max_history:
                    time_series_data[i][j].pop(0)
                time_series_data[i][j].append(val)

                # Update line
                lines[i][j].set_ydata(time_series_data[i][j])
                lines[i][j].set_xdata(x_vals[-len(time_series_data[i][j]):])

                # Optional: update heatmap text (commented)
                cell_texts[i][j].set_text(f"{val:.2f}")

        heatmap.set_data(new_matrix)

    # Return list of updated artists
    all_lines = [line for row in lines for line in row]
    return [heatmap] + [text for row in cell_texts for text in row] + all_lines
    return all_lines

def read_arduino_data():
    while True:
        try:
            line = ser.readline().decode('utf-8', errors='ignore').strip()
            if "Matrix updated:" in line:
                new_matrix = np.zeros((5, 3))
                for i in range(5):
                    row_data = ser.readline().decode('utf-8', errors='ignore').strip().split(",")
                    if len(row_data) == 3:
                        new_matrix[i] = [float(val) for val in row_data]
                data_queue.put(new_matrix)
        except Exception as e:
            print(f"Serial read error: {e}")

# Background serial reader
data_thread = threading.Thread(target=read_arduino_data, daemon=True)
data_thread.start()

# Animation loop
ani = animation.FuncAnimation(fig, update_plot, interval=50, blit=False)
plt.tight_layout()
plt.show()
